In [14]:
import polars as pl
import os
import datetime

# Download and extract GTFS data from IDFM

In [15]:
# GTFS data (used for timetable)
STATIC_GTFS_URL="https://eu.ftp.opendatasoft.com/stif/GTFS/IDFM-gtfs.zip"
STATIC_GTFS_FILE_PATH="raw_data/gtfs.zip"
STATIC_GTFS_PATH="raw_data/gtfs"

In [4]:
!mkdir -p $STATIC_GTFS_PATH
!wget $STATIC_GTFS_URL -O $STATIC_GTFS_FILE_PATH
!unzip -o $STATIC_GTFS_FILE_PATH -d $STATIC_GTFS_PATH

--2024-08-07 21:12:49--  https://eu.ftp.opendatasoft.com/stif/GTFS/IDFM-gtfs.zip
Resolving eu.ftp.opendatasoft.com (eu.ftp.opendatasoft.com)... 52.18.186.238
Connecting to eu.ftp.opendatasoft.com (eu.ftp.opendatasoft.com)|52.18.186.238|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 101360568 (97M) [application/zip]
Saving to: ‘raw_data/gtfs.zip’

raw_data/gtfs.zip   100%[===================>]  96,66M  34,3MB/s    in 2,8s    

2024-08-07 21:12:52 (34,3 MB/s) - ‘raw_data/gtfs.zip’ saved [101360568/101360568]

Archive:  raw_data/gtfs.zip
  inflating: raw_data/gtfs/transfers.txt  
  inflating: raw_data/gtfs/ticketing_deep_links.txt  
  inflating: raw_data/gtfs/agency.txt  
  inflating: raw_data/gtfs/calendar_dates.txt  
  inflating: raw_data/gtfs/calendar.txt  
  inflating: raw_data/gtfs/stops.txt  
  inflating: raw_data/gtfs/trips.txt  
  inflating: raw_data/gtfs/routes.txt  
  inflating: raw_data/gtfs/stop_extensions.txt  
  inflating: raw_data/gtfs/stop_times.

# Loading and parsing GTFS data

In [16]:
calendar_path = os.path.join(STATIC_GTFS_PATH, 'calendar.txt')
calendar_dtype = {'service_id': pl.Categorical,
                  'monday': pl.Int16,
                  'tuesday': pl.Int16,
                  'wednesday': pl.Int16,
                  'thursday': pl.Int16,
                  'friday': pl.Int16,
                  'saturday': pl.Int16,
                  'sunday': pl.Int16,
                  'start_date': pl.String,
                  'end_date': pl.String
                  }
calendar = pl.scan_csv(calendar_path, schema_overrides=calendar_dtype).select(list(calendar_dtype.keys()))
calendar = (
    calendar
    .with_columns(pl.col('monday').cast(pl.Boolean))
    .with_columns(pl.col('tuesday').cast(pl.Boolean))
    .with_columns(pl.col('tuesday').cast(pl.Boolean))
    .with_columns(pl.col('tuesday').cast(pl.Boolean))
    .with_columns(pl.col('wednesday').cast(pl.Boolean))
    .with_columns(pl.col('thursday').cast(pl.Boolean))
    .with_columns(pl.col('friday').cast(pl.Boolean))
    .with_columns(pl.col('saturday').cast(pl.Boolean))
    .with_columns(pl.col('sunday').cast(pl.Boolean))
    .with_columns(pl.col("start_date").cast(pl.String).str.to_date("%Y%m%d"))
    .with_columns(pl.col("end_date").cast(pl.String).str.to_date("%Y%m%d"))
)
calendar.head().collect()

service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
cat,bool,bool,bool,bool,bool,bool,bool,date,date
"""IDFM:1""",true,true,true,true,false,false,false,2024-09-02,2024-09-05
"""IDFM:10""",true,true,false,true,false,false,false,2024-09-02,2024-09-05
"""IDFM:100""",false,false,false,false,false,true,false,2024-08-10,2024-08-31
"""IDFM:105""",false,false,true,false,false,false,false,2024-09-04,2024-09-04
"""IDFM:1000""",true,true,true,true,true,true,true,2024-08-07,2024-09-05


In [17]:
stops_path = os.path.join(STATIC_GTFS_PATH, 'stops.txt')
stops_dtype = {'stop_id': pl.String,
               'stop_code': pl.String,
               'stop_name': pl.String,
               'stop_desc': pl.String,
               'stop_lon': pl.Float64,
               'stop_lat': pl.Float64,
               'zone_id': pl.Categorical,
               'stop_url': pl.String,
               'location_type': pl.Categorical,
               'parent_station': pl.String,
               'stop_timezone': pl.Categorical,
               'level_id': pl.String,
               'wheelchair_boarding': pl.Categorical,
               'platform_code': pl.Categorical}
stops = pl.scan_csv(stops_path, schema_overrides=stops_dtype).select(list(stops_dtype.keys()))
stops = stops.drop(['stop_code',
                    'stop_desc',
                    'stop_url',
                    'location_type',
                    'parent_station',
                    'stop_timezone',
                    'level_id',
                    'wheelchair_boarding',
                    'zone_id',
                    'platform_code'])
stops.head().collect()

stop_id,stop_name,stop_lon,stop_lat
str,str,f64,f64
"""IDFM:monomodalStopPlace:46689""","""Viroflay Rive Droite""",2.167655,48.805473
"""IDFM:monomodalStopPlace:46647""","""Achères Ville""",2.077727,48.970644
"""IDFM:monomodalStopPlace:474069""","""Lozère""",2.212718,48.705912
"""IDFM:monomodalStopPlace:474082""","""Noisy-le-Grand - Mont d'Est""",2.550027,48.840885
"""IDFM:monomodalStopPlace:45102""","""Châtelet les Halles""",2.346977,48.861745


In [18]:
trips_path = os.path.join(STATIC_GTFS_PATH, 'trips.txt')
trips_dtype = {'trip_id': pl.String,
               'route_id': pl.Categorical,
               'service_id': pl.Categorical,
               'trip_headsign': pl.String,
               'trip_short_name': pl.String,
               'direction_id': pl.String,
               'wheelchair_accessible': pl.Categorical,
               'bikes_allowed': pl.Categorical}
trips = pl.scan_csv(trips_path, schema_overrides=trips_dtype).select(list(trips_dtype.keys()))
trips = trips.drop(['direction_id', 'wheelchair_accessible', 'bikes_allowed'])
trips.head().collect()

trip_id,route_id,service_id,trip_headsign,trip_short_name
str,cat,cat,str,str
"""IDFM:KABS:154812-C00306-5940-2…","""IDFM:C00306""","""IDFM:1""","""Gare de Houilles - Carrières-s…",null
"""IDFM:TRANSDEV_MARNE_LA_VALLEE:…","""IDFM:C00638""","""IDFM:1""","""Gare de Marne-la-Vallée Chessy…",null
"""IDFM:TRANSDEV_SENART:150000-C0…","""IDFM:C00862""","""IDFM:1""","""Clos du Roi""",null
"""IDFM:TRANSDEV_SENART:150000-C0…","""IDFM:C00862""","""IDFM:1""","""Clos du Roi""",null
"""IDFM:TRANSDEV_SENART:150000-C0…","""IDFM:C00862""","""IDFM:1""","""Gare de Combs-la-Ville Quincy""",null


In [19]:
stop_times_path = os.path.join(STATIC_GTFS_PATH, 'stop_times.txt')
stop_times_dtype = {'trip_id': pl.String,
                    'arrival_time': pl.String,
                    'departure_time': pl.String,
                    'stop_id': pl.String,
                    'stop_sequence': pl.Int16
                    }
stop_times = pl.scan_csv(stop_times_path, schema_overrides=stop_times_dtype).select(list(stop_times_dtype.keys()))

# Parse departure and arrival times
stop_times = (
    stop_times.with_columns(
        pl.col('arrival_time')
        .str.splitn(":", 3)
        .struct.rename_fields(["arrival_hour", "arrival_minute", "arrival_second"])
        .alias("arrival_time_fields")
    )
    .unnest("arrival_time_fields")
    .with_columns(pl.col('arrival_hour').cast(pl.Int16))
    .with_columns(pl.col('arrival_minute').cast(pl.Int16))
    .with_columns(pl.col('arrival_second').cast(pl.Int16))
)
stop_times = (
    stop_times.with_columns(
        pl.col('departure_time')
        .str.splitn(":", 3)
        .struct.rename_fields(["departure_hour", "departure_minute", "departure_second"])
        .alias("departure_time_fields")
    )
    .unnest("departure_time_fields")
    .with_columns(pl.col('departure_hour').cast(pl.Int16))
    .with_columns(pl.col('departure_minute').cast(pl.Int16))
    .with_columns(pl.col('departure_second').cast(pl.Int16))
)

stop_times = stop_times.drop(['arrival_time', 'departure_time'])

stop_times.head().collect()

trip_id,stop_id,stop_sequence,arrival_hour,arrival_minute,arrival_second,departure_hour,departure_minute,departure_second
str,str,i16,i16,i16,i16,i16,i16,i16
"""IDFM:KABS:154812-C00306-5940-2…","""IDFM:7511""",0,14,15,0,14,15,0
"""IDFM:KABS:154812-C00306-5940-2…","""IDFM:7465""",1,14,15,0,14,15,0
"""IDFM:KABS:154812-C00306-5940-2…","""IDFM:7520""",2,14,16,0,14,16,0
"""IDFM:KABS:154812-C00306-5940-2…","""IDFM:7451""",3,14,18,0,14,18,0
"""IDFM:KABS:154812-C00306-5940-2…","""IDFM:7455""",4,14,19,0,14,19,0


In [20]:
routes_path = os.path.join(STATIC_GTFS_PATH, 'routes.txt')
routes_dtype = {'route_id': pl.Categorical,
                'agency_id': pl.Categorical,
                'route_short_name': pl.String,
                'route_long_name': pl.String,
                'route_type': pl.Categorical}
routes = pl.scan_csv(routes_path, schema_overrides=routes_dtype).select(list(routes_dtype.keys()))

routes = routes.with_columns(
    pl
    .when(pl.col('route_type') == '0').then(pl.lit('TRAM'))
    .when(pl.col('route_type') == '1').then(pl.lit('METRO'))
    .when(pl.col('route_type') == '2').then(pl.lit('TRAIN'))
    .when(pl.col('route_type') == '3').then(pl.lit('BUS'))

    .alias('route_type')
    .cast(pl.Categorical)
)

routes.head().collect()

route_id,agency_id,route_short_name,route_long_name,route_type
cat,cat,str,str,cat
"""IDFM:C01624""","""IDFM:1080""","""409""","""409""","""BUS"""
"""IDFM:C02313""","""IDFM:1081""","""01""","""01""","""BUS"""
"""IDFM:C00029""","""IDFM:81""","""502""","""502""","""BUS"""
"""IDFM:C00632""","""IDFM:64""","""2234""","""2234""","""BUS"""
"""IDFM:C00164""","""IDFM:1081""","""3""","""3""","""BUS"""


# Join all data

In [21]:
timetable = (
    stop_times
    .join(trips, on='trip_id')
    .join(calendar, on='service_id')
    .join(stops, on='stop_id')
    .join(routes, on='route_id')
)

In [22]:
# Parse times
# For trips ending after midnight, the start_date/end_date of the stop times dataframe is set the day before.

timetable = (
    timetable

    # Change start/end date to set it one day in the future
    .with_columns(
        pl
        .when(pl.col('arrival_hour') >= 24)
        .then(pl.col('start_date') + datetime.timedelta(days=1))
        .otherwise(pl.col('start_date'))
        .alias('start_date')
    )
    .with_columns(
        pl
        .when(pl.col('arrival_hour') >= 24)
        .then(pl.col('end_date') + datetime.timedelta(days=1))
        .otherwise(pl.col('end_date'))
        .alias('end_date')
    )

    # Change the day booleans to set it to the value of the day before
    .with_columns(
        pl
        .when(pl.col('arrival_hour') >= 24)
        .then(pl.col('saturday'))
        .otherwise(pl.col('sunday'))
        .alias('sunday')
    )
    .with_columns(
        pl
        .when(pl.col('arrival_hour') >= 24)
        .then(pl.col('friday'))
        .otherwise(pl.col('saturday'))
        .alias('saturday')
    )
    .with_columns(
        pl
        .when(pl.col('arrival_hour') >= 24)
        .then(pl.col('thursday'))
        .otherwise(pl.col('friday'))
        .alias('friday')
    )
    .with_columns(
        pl
        .when(pl.col('arrival_hour') >= 24)
        .then(pl.col('wednesday'))
        .otherwise(pl.col('thursday'))
        .alias('thursday')
    )
    .with_columns(
        pl
        .when(pl.col('arrival_hour') >= 24)
        .then(pl.col('tuesday'))
        .otherwise(pl.col('wednesday'))
        .alias('wednesday')
    )
    .with_columns(
        pl
        .when(pl.col('arrival_hour') >= 24)
        .then(pl.col('monday'))
        .otherwise(pl.col('tuesday'))
        .alias('tuesday')
    )
    .with_columns(
        pl
        .when(pl.col('arrival_hour') >= 24)
        .then(pl.col('sunday'))
        .otherwise(pl.col('monday'))
        .alias('monday')
    )

    # Finally, convert hours after midnight to real hours
    .with_columns(
        pl
        .when(pl.col('arrival_hour') >= 24)
        .then(pl.col('arrival_hour') - 24)
        .otherwise(pl.col('arrival_hour'))
        .alias('arrival_hour')
    )
    .with_columns(
        pl
        .when(pl.col('departure_hour') >= 24)
        .then(pl.col('departure_hour') - 24)
        .otherwise(pl.col('departure_hour'))
        .alias('departure_hour')
    )

    # Parse time component into time dtype
    .with_columns(
        pl.time(
            pl.col('arrival_hour'),
            pl.col('arrival_minute'),
            pl.col('arrival_second'),
            0)
        .alias('arrival_time')
    )
    .with_columns(
        pl.time(
            pl.col('departure_hour'),
            pl.col('departure_minute'),
            pl.col('departure_second'),
            0)
        .alias('departure_time')
    )

    .drop([
        'departure_hour',
        'departure_minute',
        'departure_second',
        'arrival_hour',
        'arrival_minute',
        'arrival_second'])
)

# Export timetable

In [23]:
timetable.collect().write_parquet('data/timetable.parquet')

In [24]:
timetable.head().collect()

trip_id,stop_id,stop_sequence,route_id,service_id,trip_headsign,trip_short_name,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date,stop_name,stop_lon,stop_lat,agency_id,route_short_name,route_long_name,route_type,arrival_time,departure_time
str,str,i16,cat,cat,str,str,bool,bool,bool,bool,bool,bool,bool,date,date,str,f64,f64,cat,str,str,cat,time,time
"""IDFM:KABS:154812-C00306-5940-2…","""IDFM:7511""",0,"""IDFM:C00306""","""IDFM:1""","""Gare de Houilles - Carrières-s…",null,true,true,true,true,false,false,false,2024-09-02,2024-09-05,"""Marché des Côteaux""",2.233614,48.961037,"""IDFM:1054""","""34""","""34""","""BUS""",14:15:00,14:15:00
"""IDFM:KABS:154812-C00306-5940-2…","""IDFM:7465""",1,"""IDFM:C00306""","""IDFM:1""","""Gare de Houilles - Carrières-s…",null,true,true,true,true,false,false,false,2024-09-02,2024-09-05,"""Craonne""",2.231505,48.960501,"""IDFM:1054""","""34""","""34""","""BUS""",14:15:00,14:15:00
"""IDFM:KABS:154812-C00306-5940-2…","""IDFM:7520""",2,"""IDFM:C00306""","""IDFM:1""","""Gare de Houilles - Carrières-s…",null,true,true,true,true,false,false,false,2024-09-02,2024-09-05,"""Passemay""",2.225993,48.960187,"""IDFM:1054""","""34""","""34""","""BUS""",14:16:00,14:16:00
"""IDFM:KABS:154812-C00306-5940-2…","""IDFM:7451""",3,"""IDFM:C00306""","""IDFM:1""","""Gare de Houilles - Carrières-s…",null,true,true,true,true,false,false,false,2024-09-02,2024-09-05,"""Route de Cormeilles""",2.231836,48.955647,"""IDFM:1054""","""34""","""34""","""BUS""",14:18:00,14:18:00
"""IDFM:KABS:154812-C00306-5940-2…","""IDFM:7455""",4,"""IDFM:C00306""","""IDFM:1""","""Gare de Houilles - Carrières-s…",null,true,true,true,true,false,false,false,2024-09-02,2024-09-05,"""Rue de Coudray""",2.230188,48.954495,"""IDFM:1054""","""34""","""34""","""BUS""",14:19:00,14:19:00
